In [49]:
import xml.etree.ElementTree as ET
import datetime
import time
import sys
from urllib.parse import urlencode
from urllib.request import urlopen
from urllib.error import HTTPError
import pandas as pd



In [11]:
tree1 = ET.parse('/content/query.xml')

In [12]:
root1 = tree1.getroot()

In [13]:
for child in root1:
  # print("Tag: ")
  print (child.tag)
  # print("\t Attribute: ")
  print(child.attrib)

{http://www.w3.org/2005/Atom}link
{'href': 'http://arxiv.org/api/query?search_query%3Dau%3Adel_maestro%20ANDNOT%20%28ti%3Acheckerboard%20OR%20ti%3APyrochlore%29%26id_list%3D%26start%3D0%26max_results%3D10', 'rel': 'self', 'type': 'application/atom+xml'}
{http://www.w3.org/2005/Atom}title
{'type': 'html'}
{http://www.w3.org/2005/Atom}id
{}
{http://www.w3.org/2005/Atom}updated
{}
{http://a9.com/-/spec/opensearch/1.1/}totalResults
{}
{http://a9.com/-/spec/opensearch/1.1/}startIndex
{}
{http://a9.com/-/spec/opensearch/1.1/}itemsPerPage
{}
{http://www.w3.org/2005/Atom}entry
{}
{http://www.w3.org/2005/Atom}entry
{}
{http://www.w3.org/2005/Atom}entry
{}
{http://www.w3.org/2005/Atom}entry
{}
{http://www.w3.org/2005/Atom}entry
{}
{http://www.w3.org/2005/Atom}entry
{}
{http://www.w3.org/2005/Atom}entry
{}
{http://www.w3.org/2005/Atom}entry
{}
{http://www.w3.org/2005/Atom}entry
{}
{http://www.w3.org/2005/Atom}entry
{}


In [3]:
newURL = '{http://arxiv.org/OAI/arXiv/}'
W3 = '{http://www.w3.org/2005/Atom}'
A9 = '{http://a9.com/-/spec/opensearch/1.1/}'
myBASE = 'http://export.arxiv.org/api/query?'

In [4]:
url = 'http://export.arxiv.org/api/query?search_query=all:network'
response = urlopen(url)
xml = response.read()

In [5]:
root = ET.fromstring(xml)
records = root.findall(W3 + 'entry')

In [6]:
root.tag

'{http://www.w3.org/2005/Atom}feed'

In [7]:
print(records)

[<Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a5285e8>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a528bd8>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a536368>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a536908>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a536e08>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a0984a8>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a098bd8>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a09d368>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a09dc28>, <Element '{http://www.w3.org/2005/Atom}entry' at 0x7ff61a0a64f8>]


In [53]:
field = ['id','url','summary','title','author','updated']
  # with open('scrape.csv', 'w') as csvfile:
  #   writer = csv.DictWriter(csvfile, fieldnames=field)
  #   writer.writeheader()
  #   writer.writerows(dict)
df_op = pd.DataFrame()
# print(output.head())
for record in records:
  meta = Record(record).output()
  df_op = df_op.append(meta, ignore_index=True)


df_op.to_csv("scrape.csv",index=False)
 


In [45]:
class Record(object):

  def __init__(self, xml_file):
    self.xml = xml_file
    self.id = self._get_text(W3, 'id')
    self.summary = self._get_text(W3, 'summary')
    self.title = self._get_text(W3, 'title')
    self.author = self._get_authors()
    self.updated = self._get_year()
    # self.updated = self._get_text(W3, 'updated')

  def _get_text(self, namespace, tag):
    try:
      return self.xml.find(namespace + tag).text.strip().lower().replace('\n','.')
    except:
      return ''

  def _get_name(self, parent, attribute):
    try: 
      return parent.find(W3 + attribute).text.lower()
    except:
      return "n/a"

  def _get_authors(self):
    authors_xml = self.xml.findall(W3 + 'author')
    names = [self._get_name(author, 'name') for author in authors_xml]
    # first_names = [self._get_name(author, 'forenames') for author in authors_xml]
    # full_names = [a+' '+b for a,b in zip(first_names, last_names)]
    return names

  def _get_year(self):
    updated_year = self._get_text(W3, 'updated')
    year = updated_year[0:4]
    return year

  def output(self):
    dict = {
        'id' : self.id,
        'summary' : self.summary,
        'title' : self.title,
        'author' : self.author,
        'updated': self.updated
    }
    return dict